In [1]:
import duckdb
import xml.etree.ElementTree as ET
# Import classifier
from ipynb.fs.full.group_classifier import predict_batch
import spacy
from spacy.tokens import Doc
print("Import successful!!")

/Users/marisbuttmann/PycharmProjects/stance-detection-german-llm/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Import successful
Import successful!!


In [3]:
# Connect to sql database
con = duckdb.connect(database='../data/database/german-parliament', read_only=False)

In [4]:
def preprocess_speech(speech:tuple) -> tuple[int, list[str]]:
    # We only need the speech it to later identify the origin speech
    speech_id = test_speech[0]
    text = test_speech[5]
    # Parse xml from string will throw ParseError if not parseable
    root = ET.fromstring(text)
    # We only need paragraphs -> this filters unecessary information, such as comments or applause
    paragraphs = root.findall('p')
    paragraphs_text = []
    # Get text of paragraphs
    for p in paragraphs:
        # Use this to filter out speaker information
        if p.attrib.get("klasse") == "redner":
            continue
        else:
            p_text = p.text.strip()
            item = {"paragraphs":[p_text]}
            paragraphs_text.append(item) #get text and remove potential irrelevant whitespaces

    return (speech_id, paragraphs_text)
    

In [5]:
test_speech = con.sql("select * from speech order by random() limit 1 ").fetchone()

CatalogException: Catalog Error: Table with name speech does not exist!
Did you mean "pg_sequences"?

In [6]:
speech_id, paragraphs = preprocess_speech(test_speech)

print(speech_id)
print(paragraphs)

NameError: name 'test_speech' is not defined

In [7]:
paragraphs = {'paragraphs': ['Herr Kollege Erhard, es wird nachher noch meine Aufgabe sein, nachzuwei sen, daß wir daraus Lehren gezogen haben.', 'Meine Damen und Herren, nicht auf die fruchtbaren und erforderlichen Diskussionen darüber spiele ich an, ob in jenen Entwürfen der Schutz des ungeborenen Lebens und die Verbesserung des Lebensschutzes zweckentsprechend organisiert waren oder nicht. Dieses war die legitime Diskussion. Es sollte, denke ich, deutlich geworden sein, was ich mit meiner Anklage über den Stil der Auseinandersetzung gemeint habe. Angenommen, es würde sich nachhaltig erweisen, daß durch Beratung Leben zu schützen sei, die Strafbedrohung der Schwangeren aber Lebensschutz nicht nur nicht bewirken, sondern gar noch den durch die Beratung erreichbaren Schutz zunichte machen könnte, angenommen also, es würde sich erweisen, daß das Verhältnis zwischen Beratung und Strafbedrohung der Schwangeren eher ein ausschließendes ist — wir, meine Damen und Herren, würden uns dann gleichwohl nicht herbeilassen, jene zu Mordkomplizen zu erklären, die die Strafbedrohung für das gewichtigere Schutzelement gehalten haben. Uns umgekehrt ist dieses widerfahren. Wer nun meinte, meine Damen und Herren, mit der Einsicht, daß eine einfühlsame und helfende Beratung ungeborenes Lebens zu schützen und zu erhalten vermag, sei für die hier zu führende Auseinandersetzung nicht viel gewonnen, der würde verkennen, daß zwischen den beiden Elementen — einerseits des beratenden und andererseits des bestrafenden Lebensschutzes — ein unaufhebbares Spannungsverhältnis besteht. Die Formel Beratung plus Strafbedrohung gleich optimaler Lebensschutz geht nicht auf. Sie ist papieren, hat nichts mit der bestimmten, in ihrer konkreten Lage ratlosen Schwangeren zu tun, die auf das Beratungsangebot nicht eingehen wird, wenn das Ergebnis der Beratung nicht in einer für sie überschaubaren Weise offen ist. Von welcher Schwangeren ist da die Rede? Ist es, meine Damen und Herren, nicht bemerkenswert, daß kaum je vernehmlich darüber nachgedacht wurde, für wen und auf wen hin wir hier überhaupt gesetzgeberisch tätig werden wollen und sollen?! Gewiß hat das Gesetz jede und jeden zum Adressaten. Aber es ist nicht für jede Schwangere relevant. Es wird nicht gemacht für die Frauen, für die es komme, was da wolle — außer Frage steht, daß das empfangene Leben anzunehmen und auszutragen sei. Es wird jedoch letztlich, tatsächlich, im Blick auf das schützende Rechtsgut auch nicht gemacht und ohne Bedeutung sein für Frauen, die unter allen Umständen entschlossen sind, die Schwangerschaft abzubrechen. Im Zeitalter der offenen Grenzen zu anderen Ländern hin — um nur eine Ausweichmöglichkeit zu nennen — wird sich da wohl niemand irgendwelchen Illusionen hingeben. Selbst eine drastische Erhöhung des Strafrahmens, für die denn bemerkenswerterweise auch niemand plädiert hat, würde in bezug auf zum Abbruch fest entschlossene Schwangere ohne jede Wirkung sein. Für wen also machen wir die Reform des § 218 Strafgesetzbuch? Wir machen sie für jene schwangeren Frauen, die hin und hergerissen sind zwischen ihrem Gewissensanruf — auch ihrem Wunsch wahrscheinlich, das Kind auszutragen — und der Sorge um die für sie und andere zunächst unabsehbaren Folgen der Geburt des Kindes. Mit Relevanz für diese Frauen hat der Gesetzgeber tätig zu werden; sie fordern ihn: die Schwangeren, die der Beratung und der Hilfe bedürfen und die der Beratung dann zugänglich sein werden, sie dann suchen werden, wenn die Beratung — obzwar zur Annahme der Schwangerschaft ermutigend — im Ergebnis jedoch in überschaubarer Weise offen ist und jedenfalls nicht in die Strafbedrohung führt. Eine Veranstaltung, bei der auf einen anderen so eingewirkt wird, daß das Ergebnis dieser Einwirkung von vornherein feststeht, wird man schwerlich Beratung nennen können. Der strafrechtliche Schutz würde zum beratend helfenden Schutz dann eben nicht in einem komplementären, in einem flankierenden Verhältnis stehen; er bildete vielmehr die Barriere, die das im Ergebnis dem strafandrohenden Schutz überlegene Beratungsangebot ins Leere stoßen ließe. Die Formel Strafbedrohung plus Beratung gleich optimaler Lebensschutz stimmt nicht; ich sagte es schon. Sie reicht nicht an den in der Tat insoweit miserablen Zustand heran, als für das geltende Recht zutrifft, was in der Erklärung der katholischen Bischöfe der im Sinne eines wirksameren Lebensschutzes gebotenen Änderung des geltenden Rechts zugedacht ist: daß das Gesetz dem Töten einen pseudorechtlichen Freiraum gewähre. Damit ist der derzeitige Zustand beschrieben. Wer diese schlechte Wirklichkeit ändern will, muß dem aufgezeigten Spannungsverhältnis zwischen den beiden Komponenten — einerseits des helfend beratenden Lebensschutzes und andererseits des strafandrohenden Lebensschutzes — Rechnung tragen. Der vorliegende Gesetzentwurf kombiniert die beiden Elemente so, daß sie für sich Bestand haben und sich nicht gegenseitig aufheben. Er trägt den logischen und psychologischen Bedingungen Rechnung, unter denen Beratung angenommen wird. Dies gilt sowohl für die Organisation der Beratung als auch, was die Indikationen anbelangt, für das erforderliche Ausmaß überschaubarer und verantwortbarer Offenheit des von der Strafbedrohung gezogenen Entscheidungsrahmens. Frau Kollegin Dr. Neumeister, es ist doch nicht so, daß, was die Organisation der Beratung anlangt, alles auf einen Arzt abgestellt würde. Es ist doch nicht so! Der Arzt, der den Schwangerschaftsabbruch vornimmt, kann sich doch nicht einfach darauf berufen, daß er einen Wisch Papier in der Hand hat. Dies ist doch nicht so gedacht, kann doch so nicht sein.', 'Im übrigen, meine Damen und Herren, wahrt unser Entwurf die Würde der Frau. Ich weiß, daß das alles auch ganz anders gesehen werden kann. Ich weiß, daß unter dem Stichwort der sittenbildenden Kraft des Strafrechts das Bestreben darauf gerichtet sein kann, abstrakt die Norm hochzuhalten. Ich werte das nicht ab, habe aber zu wiederholen, da bei allzu hochgehaltener, in Strafbedrohung sich manifestierender Norm die konkrete Hilfe nicht ankommt. Beide Elemente lassen sich nicht beliebig addieren. Im Gegenteil: Sie können sich gegenseitig Abbruch tun. So manches in den letzten zwei Jahren auch mit mir in meinem religiösen Bekenntnis verbundenen Menschen geführte Gespräch hat sich auf die Frage hin zugespitzt, ob es um das Hochhalten der Norm oder um die Veränderung einer schlechten gesellschaftlichen Wirklichkeit geht. Beides zusammen —das ist meine These; ich meine, daß sie begründet ist — wird schwerlich zu haben sein. Gewiß, die meisten dieser meiner Gesprächspartner lehnten es ab, sich auf die Alternative in dieser Deutlichkeit einzulassen. Einige aber taten es und gaben mir zur Antwort, daß in einem solchen Konflikt das Hochhalten der Norm wichtiger sei. Was sich dann hinter einem solchen Schild abspiele, gehe auf das Konto menschlicher Unzulänglichkeiten. Hier, meine Damen und Herren, sind wir am Kern der Sache, am Kern des Problems. Wenn ich das so sage, werde ich mich gleichwohl hüten, Menschen dieser Denkart den Willen abzusprechen, ihrerseits Lebensschutz bewirken zu wollen. Aber wir sind am Kern des Problems, den mein Freund Erhard Eppler in einem das Thema übergreifenden Konflikt zwischen Wert- und Strukturkonservativismus angelegt sieht. Wer die Werte und hier den höchsten Wert — menschliches Leben — bewahren will, muß unter gewandelten gesellschaftlichen Bedingungen die Strukturen verändern. Daß sich die gesellschaftlichen Bedingungen gewandelt haben, kann doch in bezug auf unser Problem niemand in Abrede stellen, der nur daran denkt, daß heute die Reise nach Holland oder anderswohin nahezu allen erschwinglich ist. Strukturen verändern, in diesem Falle ein schlechtes, ein untaugliches, ein nicht mehr zielgemäßes Gesetz verändern, um die Werte erhalten zu können, darum geht es. Anders gesagt: Wer primär die Strukturen erhalten möchte, der wird — und sei es ungewollt — die Bewahrung der Werte gefährden. In diesem Zusammenhang und in dieser Auseinandersetzung geht es wohl auch ein wenig darum, daß es rechts angesiedelter Denkungsart gemäß ist —wenn ich mich einmal dieses unzulänglichen Schemas bedienen darf —, in die Institutionen und Regelungen größeres Zutrauen zu haben als in die Menschen, wohingegen links in die Menschen größeres Vertrauen gesetzt wird als in gesellschaftliche Apparate und Regelungen. Die bisher mit der auf Lebensschutz gerichteten Beratung der Schwangeren gemachten Erfahrungen rechtfertigen und fördern das Vertrauen in die Menschen. Dabei sehe ich natürlich auch die in der von den Koalitionsfraktionen angebotenen Lösung steckenden Risiken. Sollte das heute zu verabschiedende Gesetz nach einer angemessenen Zeit seiner Wirksamkeit nicht halten, was man sich von ihm im Sinne besserer Gewährleistung des Lebensschutzes verspricht, so würde das Thema wieder auf dem Tisch liegen. Lassen Sie mich nochmals auf einen Gedanken zurückkommen. Dieser Gesetzentwurf wahrt auch die Würde der Frau. Auf dieses Postulat wird immer Bedacht genommen werden müssen, wie immer man an die Sache herangeht. Zum Schluß möchte ich noch einmal darum bitten, damit aufzuhören, uns gegenseitig den Willen abzusprechen, ungeborenes Leben schützen zu wollen.', 'Die bisher gegen uns vorgetragenen Attacken zeugten in oft befremdlicher Weise von einer Selbstsicherheit, die niemand haben kann, der sich vor Ort am konkreten Einzelfall die Probleme ansieht. Weniger Selbstgewißheit, aber allseits mehr den festen Willen, das Beratungsangebot zu verbessern und insgesamt kinderfreundlichere Verhältnisse in diesem Land zu schaffen!']}

classified_paragraphs = predict_batch(paragraphs)

Map: 100%|██████████| 4/4 [00:00<00:00, 501.68 examples/s]


In [9]:
print(classified_paragraphs)

[('[CLS]', 'O'), ('Herr', 'B-EPPOL'), ('Kollege', 'I-EPPOL'), ('Er', 'I-EPPOL'), ('##hard', 'I-EPPOL'), (',', 'O'), ('es', 'O'), ('wird', 'O'), ('nach', 'O'), ('##her', 'O'), ('noch', 'O'), ('meine', 'O'), ('Aufgabe', 'O'), ('sein', 'O'), (',', 'O'), ('nachzu', 'O'), ('##wei', 'O'), ('sen', 'O'), (',', 'O'), ('daß', 'O'), ('wir', 'O'), ('daraus', 'O'), ('Lehre', 'O'), ('##n', 'O'), ('gezogen', 'O'), ('haben', 'O'), ('.', 'O'), ('[SEP]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]', 'O'), ('[PAD]',

In [8]:
for index, p in enumerate(classified_paragraphs):
    print(p, "\n\n")

('[CLS]', 'O') 


('Herr', 'B-EPPOL') 


('Kollege', 'I-EPPOL') 


('Er', 'I-EPPOL') 


('##hard', 'I-EPPOL') 


(',', 'O') 


('es', 'O') 


('wird', 'O') 


('nach', 'O') 


('##her', 'O') 


('noch', 'O') 


('meine', 'O') 


('Aufgabe', 'O') 


('sein', 'O') 


(',', 'O') 


('nachzu', 'O') 


('##wei', 'O') 


('sen', 'O') 


(',', 'O') 


('daß', 'O') 


('wir', 'O') 


('daraus', 'O') 


('Lehre', 'O') 


('##n', 'O') 


('gezogen', 'O') 


('haben', 'O') 


('.', 'O') 


('[SEP]', 'O') 


('[PAD]', 'O') 


('[PAD]', 'O') 


('[PAD]', 'O') 


('[PAD]', 'O') 


('[PAD]', 'O') 


('[PAD]', 'O') 


('[PAD]', 'O') 


('[PAD]', 'O') 


('[PAD]', 'O') 


('[PAD]', 'O') 


('[PAD]', 'O') 


('[PAD]', 'O') 


('[PAD]', 'O') 


('[PAD]', 'O') 


('[PAD]', 'O') 


('[PAD]', 'O') 


('[PAD]', 'O') 


('[PAD]', 'O') 


('[PAD]', 'O') 


('[PAD]', 'O') 


('[PAD]', 'O') 


('[PAD]', 'O') 


('[PAD]', 'O') 


('[PAD]', 'O') 


('[PAD]', 'O') 


('[PAD]', 'O') 


('[PAD]', 'O') 


('[PAD]', 'O

In [41]:
p = classified_paragraphs [1]
groups = []
group_tmp = []
new_group = False
for token, label in p:
    if label == 'O':
        if group_tmp:
            groups.append(group_tmp)
            group_tmp = []
        new_group = True
        continue
    if new_group:
        new_group = False
    group_tmp.append((token, label))

print(groups)

[[('der', 'B-PFUNK'), ('Schwangeren', 'I-PFUNK')], [('der', 'B-PFUNK'), ('Schwangeren', 'I-PFUNK')], [('haben', 'I-PSOZ')], [('ratlosen', 'B-PFUNK'), ('Schwangeren', 'I-PFUNK')], [('welcher', 'B-PFUNK'), ('Schwangeren', 'I-PFUNK')], [('jede', 'B-PFUNK'), ('Schwangere', 'I-PFUNK')], [('die', 'B-PSOZ'), ('Frauen', 'I-PSOZ')], [('Frauen', 'B-PSOZ'), (',', 'I-PSOZ'), ('die', 'I-PSOZ'), ('unter', 'I-PSOZ'), ('allen', 'I-PSOZ'), ('Umständen', 'I-PSOZ'), ('entschlossen', 'I-PSOZ'), ('sind', 'I-PSOZ'), (',', 'I-PSOZ'), ('die', 'I-PSOZ')], [('abzubrechen', 'I-PSOZ')]]


In [53]:
nlp = spacy.blank("de")
words = [token for token, label in groups[7]]
print(words)
doc = Doc(nlp.vocab, words=words)
text = doc.text
print(text)

['Frauen', ',', 'die', 'unter', 'allen', 'Umständen', 'entschlossen', 'sind', ',', 'die']
Frauen , die unter allen Umständen entschlossen sind , die 
